In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import (VBox, HBox, Button, Dropdown, FloatRangeSlider, 
                        Label, Layout, FloatSlider, FloatText, jslink,
                        Output)
import numpy as np

In [51]:
def get_min_max_slider(name, default_val=1, min_init=-4, max_init=4, 
                       label_text=None, range_slider=False):
    if not label_text:
        label_text = name
    min_val = FloatText(min_init, description="${}_{{min}}$".format(name.strip("$")))
    max_val = FloatText(max_init, description="${}_{{max}}$".format(name.strip("$")))
    if range_slider:
        slider = FloatRangeSlider(min=min_init, max=max_init,
                                  description="{}".format(label_text),  
                                  layout=Layout(width='500px'))
    else:
        slider = FloatSlider(value=default_val, min=min_val.value, max=max_val.value, 
                             description="{}".format(label_text), layout=Layout(width='500px'))
    jslink((slider, 'min'), (min_val, "value"))
    jslink((slider, 'max'), (max_val, "value"))
    return slider, HBox([slider, min_val, max_val])

def get_data(func_set, t_min, t_max, A, omega, phi):
    yvals = []
    t = np.linspace(t_min, t_max, num=200)
    if 0 in func_set:
        yvals.append([t, A*np.sin(omega*t+phi)])
    if 1 in func_set:
        yvals.append([t, A*np.cos(omega*t+phi)])
    if 2 in func_set:
        yvals.append([t, A*np.tan(omega*t+phi)])
    if 3 in func_set:
        yvals.append([t, A*np.exp(omega*t+phi)])
    return yvals


def interactiv_plot():
    func_set = set()
    func_names = ['A sin(ω t + φ)', 'A cos(ω t + φ)', 'A tan(ω t + φ)', 'A exp(ω t + φ)']
    options_dict = dict(map(lambda x: (x[1], x[0]), list(enumerate(func_names))))
    function_name = Dropdown(options=options_dict, description='Function:')
    add_btn = Button(icon='plus', button_style='success', tooltip="add function to be ploted")
    del_btn = Button(icon='minus', button_style='danger', tooltip="remove function to be ploted")
    
    plotrange_t, plotrange_t_box = get_min_max_slider("t", min_init=-2*np.pi, max_init=4*np.pi, 
                                                  label_text='Plotrange $t$', range_slider=True)
    
    plotrange_y, plotrange_y_box = get_min_max_slider("y", min_init=-3, max_init=3, 
                                                  label_text='Plotrange $y$', range_slider=True)
    a_slider, a_slider_box = get_min_max_slider("$A$")
    omega_slider, omega_slider_box = get_min_max_slider("$ω$")
    phi_slider, phi_slider_box = get_min_max_slider("$φ$")
    out = Output()
    
    def update_plot(val):
        out.clear_output(wait=True)
        t_min, t_max = plotrange_t.value
        y_min, y_max = plotrange_y.value
        A = a_slider.value
        omega = omega_slider.value
        phi = phi_slider.value
        
        data = get_data(func_set, t_min, t_max, A, omega, phi)
        with out:
            for func_data, func_index in zip(data, func_set):
                plt.plot(*func_data, label=func_names[func_index])
            plt.legend(loc='upper left')
            plt.title("The chosen Parameters are:\n" \
                      "A={}, ω={} and φ={}" \
                      "".format(A, omega, phi))
            axes = plt.gca()
            axes.set_ylim([y_min, y_max])
            axes.set_xlabel("t")
            axes.set_ylabel("y")
            plt.show()
    
    plotrange_t.observe(update_plot, names='value')
    plotrange_y.observe(update_plot, names='value')
    a_slider.observe(update_plot, names='value')
    omega_slider.observe(update_plot, names='value')
    phi_slider.observe(update_plot, names='value')
        
    def add_func(btn):
        func_set.add(function_name.value)
        with out:
            update_plot("")
        
    add_btn.on_click(add_func)
    
    def del_func(btn):
        func_set.remove(function_name.value)
        with out:
            update_plot("")
        
    del_btn.on_click(del_func)
    
    return VBox([HBox([function_name, add_btn, del_btn]),
                 plotrange_t_box,
                 plotrange_y_box,
                 a_slider_box,
                 omega_slider_box,
                 phi_slider_box,
                 out
                ])
interactiv_plot()